In [113]:
import pyodbc
import pandas as pd
import time
from datetime import datetime

In [114]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=FREYJA\SQLEXPRESS;'
                      'Database=tradingCrypto;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM [dbo].[cryptoPrices_raw]')

for i in cursor:
    print(i)

('1641574909884', 'btc', '42055.59232485777')
('1642132899597', 'btc', '42780.31638262786')
('1642690904488', 'btc', '43120.58908265619')
('1643252493565', 'btc', '36104.216386999695')
('1643806833503', 'btc', '38778.65338070416')
('1644364906929', 'btc', '44184.447511676175')
('1644937275868', 'btc', '44324.61973817016')
('1645491639656', 'btc', '37280.91362125845')
('1646046140895', 'btc', '38402.12152228042')
('1646600409778', 'btc', '38933.71815378388')
('1647154928948', 'btc', '39181.76778596648')
('1647713001989', 'btc', '41905.176694056885')
('1648270964663', 'btc', '44428.307777978145')
('1648274623123', 'btc', '44470.532652704656')
('1648278209508', 'btc', '44406.13971170422')
('1648281781720', 'btc', '44532.89405010229')
('1648285329518', 'btc', '44478.44586047613')
('1648288813103', 'btc', '44466.53057502017')
('1648292462326', 'btc', '44415.71982853605')
('1648296139335', 'btc', '44369.8755722968')
('1648299870403', 'btc', '44359.453445398416')
('1648303288565', 'btc', '442

C:\Users\katze\AppData\Local\Temp\ipykernel_66996\245314679.py:10: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(i)


In [115]:
query = "SELECT * FROM [dbo].[cryptoPrices_raw];"
dfStg = pd.read_sql(query, conn)

print(dfStg.head(5))

      timestampp id_coin            priceUSD
0  1641574909884     btc   42055.59232485777
1  1642132899597     btc   42780.31638262786
2  1642690904488     btc   43120.58908265619
3  1643252493565     btc  36104.216386999695
4  1643806833503     btc   38778.65338070416


C:\Users\katze\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [116]:
# Add a column for the primary key
dfStg = dfStg.reset_index()
dfStg = dfStg.rename(columns={'index':'id'})  
dfStg['id'] = dfStg.index + 1
print(dfStg)

        id     timestampp id_coin            priceUSD
0        1  1641574909884     btc   42055.59232485777
1        2  1642132899597     btc   42780.31638262786
2        3  1642690904488     btc   43120.58908265619
3        4  1643252493565     btc  36104.216386999695
4        5  1643806833503     btc   38778.65338070416
...    ...            ...     ...                 ...
2155  2156  1647694944533     btc   41835.23488199481
2156  2157  1647698431167     btc  41766.747032085375
2157  2158  1647702156740     btc  41823.786665859385
2158  2159  1647705726151     btc  41974.049306458786
2159  2160  1647709316947     btc   42005.28015313712

[2160 rows x 4 columns]


In [117]:
print(dfStg.dtypes)

id             int64
timestampp    object
id_coin       object
priceUSD      object
dtype: object


In [118]:
# Transforming column timestampp and priceUSD
#dfStg["priceUSD"] = pd.to_numeric(df["a"])
dfStg = dfStg.astype({'priceUSD':'float'})
dfStg['timestampp'] = dfStg['timestampp'].str[:10]
dfStg['timestampp'] = pd.to_datetime(dfStg['timestampp'],unit='s')
dfStg.sort_values(by='timestampp') 
print(dfStg)
 

        id          timestampp id_coin      priceUSD
0        1 2022-01-07 17:01:49     btc  42055.592325
1        2 2022-01-14 04:01:39     btc  42780.316383
2        3 2022-01-20 15:01:44     btc  43120.589083
3        4 2022-01-27 03:01:33     btc  36104.216387
4        5 2022-02-02 13:00:33     btc  38778.653381
...    ...                 ...     ...           ...
2155  2156 2022-03-19 13:02:24     btc  41835.234882
2156  2157 2022-03-19 14:00:31     btc  41766.747032
2157  2158 2022-03-19 15:02:36     btc  41823.786666
2158  2159 2022-03-19 16:02:06     btc  41974.049306
2159  2160 2022-03-19 17:01:56     btc  42005.280153

[2160 rows x 4 columns]


In [119]:
print(dfStg.dtypes)

id                     int64
timestampp    datetime64[ns]
id_coin               object
priceUSD             float64
dtype: object


In [120]:
# test
unix_timestamp= int('1641020554')
utc_time = datetime.utcfromtimestamp(unix_timestamp)
#print(utc_time.strftime("%Y-%m-%d %H:%M:%S.%f+00:00 (UTC)"))
print(utc_time)


2022-01-01 07:02:34


In [121]:
# Insert Dataframe into SQL Server:
for index, row in dfStg.iterrows():
     cursor.execute("INSERT INTO dbo.cryptoPrices_st (id,timestampp,id_coin,priceUSD) values(?,?,?,?)", row.id, row.timestampp, row.id_coin, row.priceUSD)
conn.commit()
cursor.close()
# see total time to do insert
#print("%s seconds ---" % (time.time() - start_time))